---
title: "Sentiment Analysis"
format:
    html: default
    ipynb: default
execute:
  eval: false
jupyter: python3
resources:
    - "*.csv"
listing:
    id: mod-listings
    contents:
        - "*.csv"
    fields: [filename]
---

::: {.callout-tip title="Data" collapse="true"}

:::: {#mod-listings}
::::

:::

The ultimate goal of this exercise consists performing the same exercise, namely sentiment analysis, 
using traditional NLP and GPT-4.

## The Dataset

We use the [News Sentiment Dataset](https://www.kaggle.com/datasets/yasserh/twitter-tweets-sentiment-dataset) from Kaggle.

1. __Import Dataset as a pandas dataframe__

In [ ]:
# your code here (don't forget to change directory)

2. __Describe Dataset (text and graphs)__

In [ ]:
# your code here

3. __Split Dataset into training, validation and test set. What is the purpose of the validation set?__

In [ ]:
# your code here

## Text Mining

1. __Extract features from the training dataset. What do you do with non-words / punctuation?__

In [ ]:
# your code here

2. __Convert occurrencies to frequencies. Make another version with tf-idf.__

In [ ]:
# your code here

3. __Choose a classifier to predict the sentiment on the *validation* set. Compute the confusion matrix.__

In [ ]:
# your code here

## Sentiment Analysis using GPT completion

1. __Setup an openai key. Explore openai *completion* API.__

In [7]:
# make sure we have the right version
from openai import version
openai.version.VERSION

'1.55.3'

In [ ]:
####
# google: openai python api
# github page: copy first example

In [ ]:
api_key = 

In [9]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=api_key,  # This is the default and can be omitted
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-4o",
)

In [ ]:
# your code here

2. __Design a prompt to extract the sentiment from a tweet. Test it on very few tweets from the training dataset. Propose different versions.__

In [ ]:
# your code here

3. __Write a function which takes in: the prompt template, the tweet text and returns the sentiment as an integer.__

In [ ]:
# your code here

4. __Compare the results from GPT with those from NLP (on a small subset of the data)__

In [ ]:
# your code here